<a href="https://colab.research.google.com/github/AlejoMarinsalda/Python-Data/blob/main/ApiLol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from time import sleep
import pandas as pd
from datetime import datetime
import os 


In [ ]:
api_key = "RGAPI-69560027-3975-44a0-a1df-717f5ad91963"

In [ ]:
servers ={
    "br1":"br1",
     "eun1":"eun1",
     "euw1":"euw1",
     "la1":"la1",
     "la2":"la2",
     "na1":"na1",
     "oc1":"oc1",
     "ru":"ru",
     "tr1":"tr1",
}

In [ ]:
name = "alemsd14"
user_per_server = {}
for server in servers:
    endpoint=f"https://{server}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key=RGAPI-69560027-3975-44a0-a1df-717f5ad91963"
    res = requests.get(endpoint).json()
    if res.get("status", None):
        print(f"No user in {server}:{res}")
    else:
        print(f"User found in {server}")
        user_per_server[server] = res
    sleep(1)


In [ ]:
user_per_server

##Account

In [ ]:
endpoint=f"https://la2.api.riotgames.com/lol/summoner/v4/summoners/by-name/alemsd14?api_key=RGAPI-69560027-3975-44a0-a1df-717f5ad91963"
summoner = requests.get(endpoint).json()

In [ ]:
df=pd.DataFrame([summoner])
df

In [ ]:
summ_id = "qjeK5zC-LurB2kx56qh3aJvsQjvkmHoFEQKrP5idLclf7xY" 

##Extract and concat ranking mastery

In [ ]:
summ_id=summoner["id"]
endpoint=f"https://la2.api.riotgames.com/lol/league/v4/entries/by-summoner/{summ_id}?api_key=RGAPI-69560027-3975-44a0-a1df-717f5ad91963"
res = requests.get(endpoint).json()
df_rank = pd.DataFrame([res[0]])
df_rank

In [ ]:
df_summ = pd.concat([df,df_rank],axis=1)

In [ ]:
df_summ.dtypes

In [ ]:
df_summ=df_summ.drop(columns=["summonerId","summonerName"])
df_summ

In [ ]:
#lets transform revisionDate from epoch to datetime
df["revisionDate"] = pd.to_datetime(df["revisionDate"],unit="ms")
df["revisionDate"]

Save the data into a file

In [ ]:

df.to_csv(f"data/summoner.csv",index = False)

##Champions mastery

In [ ]:

endpoint=f"https://la2.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/{summ_id}?api_key=RGAPI-69560027-3975-44a0-a1df-717f5ad91963"
res = requests.get(endpoint).json()
df_champs =pd.DataFrame(res)
df_champs

In [ ]:
df_champs["lastPlayTime"] = pd.to_datetime(df_champs["lastPlayTime"],unit="ms")
df.to_csv(f"data/champs_mastery_{datetime.today()}.csv",index = False)


##Match history

In [ ]:
puuid=summoner["puuid"]
endpoint =f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/8d3WIsnmge6BGLXHsRNbV1hW6itmW_60bDRGARpFTxZBvdjbDL2PMG9MuuWxnl-l7X5OPAqbhwGxxQ../ids?start=0&count=20&api_key=RGAPI-69560027-3975-44a0-a1df-717f5ad91963"
res = requests.get(endpoint).json()


In [ ]:
res

In [ ]:
match_id=res[0]
endpoint =f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key=RGAPI-69560027-3975-44a0-a1df-717f5ad91963"
res = requests.get(endpoint).json()
res

In [ ]:
basics = [
    'puuid',
     "win",
     "lane",
      "role",
      "teamPosition",
      "kills",
      "deaths",
      "totalMinionsKilled",
      'eligibleForProgression',
       "timePlayed",
      'gameEndedInEarlySurrender',
         'gameEndedInSurrender',


  ]

kills =[
      "killingSprees",
      "firstBloodAssist",
      "firstBloodKill",
      "doubleKills",
      "tripleKills",
      "quadraKills",
       "pentaKills",
  ]

wards=[
      "wardsKilled",
      "wardsPlaced",
      "visionWardsBoughtInGame",
      "detectorWardsPlaced",
  ]

objetives = [
      "objectivesStolen",
      "turretKills",
      "dragonKills",
      "inhibitorKills",
      "nexusKills",
  ]

pings =[
      "dangerPings",
      "enemyMissingPing",
      "enemyVisionPings",
      "getBackPings",
      "holdPings",
      "needVisionPings",
      "onMyWayPings",
      "pushPings",
      "visionClearedPings",
  ]

stats = basics + kills +wards + objetives

In [ ]:
df = pd.DataFrame(res["info"]["participants"])
df = df[stats]
df